In [ ]:
!wget https://www.dropbox.com/s/9sakh5x8h1rrrip/HW4.zip

In [ ]:
!mkdir data
!unzip HW4.zip -d ./data/

In [ ]:
!mkdir data/F22
!mkdir data/F22-out
!ffmpeg -i data/F22.mp4 -vf fps=25 data/F22/out%d.png

In [ ]:
import cv2
import numpy as np
from glob import glob
from scipy import ndimage
import matplotlib.pyplot as plt

In [ ]:
def normalize(img):
    img = img/img.max()
    return img

In [ ]:
def SobelFilter(img, direction):
    if (direction == 'x'):
        Gx = np.array([[-1,0,+1], [-2,0,+2],  [-1,0,+1]])
        Res = ndimage.convolve(img, Gx)
    
    if (direction == 'y'):
        Gy = np.array([[-1,-2,-1], [0,0,0], [+1,+2,+1]])
        Res = ndimage.convolve(img, Gy)
    
    return Res

In [ ]:
image_paths = sorted(glob('./data/F22/*'))

for _, ip in enumerate(image_paths[:]):
    img = cv2.imread(ip, cv2.IMREAD_GRAYSCALE)
    index = ip.split('out')[-1]

    img = cv2.GaussianBlur(img, (9, 9), 2)
    edges = cv2.Canny(img, 50, 150)
    edges[:, :40] = 0

    # Create a green border
    img = cv2.imread(ip, cv2.IMREAD_COLOR)
    blue_c = img[:, :, 0]
    green_c = img[:, :, 1]
    red_c = img[:, :, 2]
    blue_c[edges == 255] = 0
    green_c[edges == 255] = 255
    red_c[edges == 255] = 0
    img[:, :, 0] = blue_c
    img[:, :, 1] = green_c
    img[:, :, 2] = red_c

    cv2.imwrite(f'./data/F22-out/out{index}', img)

    # fig = plt.figure(figsize=(16, 8))
    # ax0 = plt.subplot(1, 2, 1)
    # ax0.axis('off')
    # ax0.imshow(img, cmap='gray')
    # ax1 = plt.subplot(1, 2, 2)
    # ax1.axis('off')

    # ax1.imshow(edges, cmap='gray')
    # plt.show()

In [ ]:
!ffmpeg -framerate 25 -i ./data/F22-out/out%01d.png f22_track.mp4